In [10]:
import numpy as np
from keras.datasets import boston_housing
from keras import models
from keras import layers
from keras import losses
from keras import metrics
from keras import optimizers

In [11]:
(trainData, trainTargets), (testData, testTargets) = boston_housing.load_data()

trainData.shape

(404, 13)

In [12]:
#Normalize data
mean = trainData.mean(axis=0)
trainData -= mean
std = trainData.std(axis=0)
trainData /= std

testData -= mean
testData /= std

In [14]:
def buildModel():
    model = models.Sequential()
    model.add(layers.Dense(64, activation="relu", input_shape=(trainData.shape[1],)))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(1))
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    return model


In [17]:
k = 4
numValSamples = len(trainData) // k
numEpochs = 100
allScores = []

for i in range(k):
    print("processing fold #", i)
    valData = trainData[i * numValSamples: (i + 1) * numValSamples]
    valTargets = trainTargets[i * numValSamples: (i + 1) * numValSamples]
    
    partialTrainData = np.concatenate(
        [trainData[:i * numValSamples],
         trainData[(i + 1) * numValSamples:]],
        axis = 0)
    partialTrainTargets = np.concatenate(
        [trainTargets[:i * numValSamples],
         trainTargets[(i + 1) * numValSamples:]],
        axis = 0)
    
    model = buildModel()
    model.fit(partialTrainData, partialTrainTargets, epochs = numEpochs, batch_size = 1, verbose = 0)
    valMse, valMae = model.evaluate(valData, valTargets, verbose = 0)
    allScores.append(valMae)
    


processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
